In [1]:
# NOTE: PLEASE MAKE SURE YOU ARE RUNNING THIS IN A PYTHON3 ENVIRONMENT
import tensorflow as tf
print(tf.__version__)

# This is needed for the iterator over the data
# But not necessary if you have TF 2.0 installed
#!pip install tensorflow==2.0.0-beta0
#tf.enable_eager_execution()
# !pip install -q tensorflow-datasets

2.0.0


In [2]:
# Import dataset, IMDB ratings movies dataset
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info = True, as_supervised = True)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /Users/nicolasbreton/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


In [24]:
import numpy as np

# Get ttaining and test data
train_data, test_data = imdb['train'], imdb['test']

In [25]:
# Initialize training lists
training_sentences = []
training_labels = []


# Affect the sentences and the labels from the train_data
# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())

# Convert the training labels in an array
training_labels_final = np.array(training_labels)

In [26]:
# Do the same for test set
testing_sentences = []
testing_labels = []

for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

testing_labels_final = np.array(testing_labels)

In [27]:
# Define the parameters for the preprocessing

# Size of the vocabulary dictionary
vocab_size = 10000

# Size of the embedding dimension
embedding_dim = 16

# Max length of the sentences
max_length = 120

# Type of truncature
trunc_type='post'

# Token for out of context
oov_tok = "<OOV>"

In [28]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [29]:
# Initialize the tokenizer
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)

# Fit the tokenizer on the list of sentences
tokenizer.fit_on_texts(training_sentences)

# Define the word dictionary for the training sentences
word_index = tokenizer.word_index

# Transform the training sentences in training sequences of indices
sequences = tokenizer.texts_to_sequences(training_sentences)

# Pad the training sequences of indices to have uniform training data
padded = pad_sequences(sequences, maxlen = max_length, truncating = trunc_type)

# Transform the testing sentences in test sequences of indices 
# NB: while using the tokenizer fitted on the training sentences
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)

# Pad the testing sequences
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [30]:
# Define a reverse dictionary to map from the indices to the words
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# Decode a review from a padded format
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print('sentence use for the exemple:\n', training_sentences[1])
print('\n')
print('text padded:\n ', padded[1])
print('\n')
print('decode text padded:\n', decode_review(padded[1]))

sentence use for the exemple:
 b'I have no idea what the other reviewer is talking about- this was a wonderful movie, and created a sense of the era that feels like time travel. The characters are truly young, Mary is a strong match for Byron, Claire is juvenile and a tad annoying, Polidori is a convincing beaten-down sycophant... all are beautiful, curious, and decadent... not the frightening wrecks they are in Gothic.<br /><br />Gothic works as an independent piece of shock film, and I loved it for different reasons, but this works like a Merchant and Ivory film, and was from my readings the best capture of what the summer must have felt like. Romantic, yes, but completely rekindles my interest in the lives of Shelley and Byron every time I think about the film. One of my all-time favorites.'


text padded:
  [ 256   28   61  331   51    2   85 2213    7  667   45   12   14    4
  395   18    3 1092    4  282    5    2  998   13  773   39   60 2139
    2  106   25  378  185 1083    7

In [31]:
# Define a sequence model - RNN
model = tf.keras.Sequential([
    # Embedding layer
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    # Flatten the embedding layer
    tf.keras.layers.Flatten(),
    # Dense layer
    tf.keras.layers.Dense(6, activation = 'relu'),
    # Dense layer
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [32]:
num_epochs = 10
model.fit(padded, # Training inputs
          training_labels_final, # Training outputs
          epochs = num_epochs, 
          validation_data = (testing_padded, testing_labels_final)) # Validation data = inputs + ouputs

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 10s 393us/sample - loss: 0.4919 - accuracy: 0.7446 - val_loss: 0.3522 - val_accuracy: 0.8474
Epoch 2/10
25000/25000 [==============================] - 8s 329us/sample - loss: 0.2340 - accuracy: 0.9100 - val_loss: 0.3746 - val_accuracy: 0.8374
Epoch 3/10
25000/25000 [==============================] - 10s 384us/sample - loss: 0.0840 - accuracy: 0.9794 - val_loss: 0.4521 - val_accuracy: 0.8292
Epoch 4/10
25000/25000 [==============================] - 8s 313us/sample - loss: 0.0199 - accuracy: 0.9976 - val_loss: 0.5281 - val_accuracy: 0.8271
Epoch 5/10
25000/25000 [==============================] - 8s 319us/sample - loss: 0.0048 - accuracy: 0.9997 - val_loss: 0.5985 - val_accuracy: 0.8249
Epoch 6/10
25000/25000 [==============================] - 8s 322us/sample - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.6357 - val_accuracy: 0.8279
Epoch 7/10
25000/25000 [========================

In [33]:
# Get the embedding layer
e = model.layers[0]

# Get the weights
weights = e.get_weights()[0]

# Print the shape
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [34]:
import io

# Print the embeddings vector of each word
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, 100): # vocab_size
    print(word_num)
    word = reverse_word_index[word_num]
    print(word)
    embeddings = weights[word_num]
    print(embeddings)
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
    print('\n')
out_v.close()
out_m.close()

1
<OOV>
[ 0.02166153 -0.07379112  0.00358837 -0.02598774  0.00743789  0.00348573
  0.05399512  0.01379667  0.03188702 -0.06391246 -0.04461209 -0.0310375
  0.11905223  0.00799899 -0.00416816 -0.00624377]


2
the
[-0.01059231 -0.04134747  0.00478015  0.04746634 -0.02882392 -0.04474274
  0.04731398  0.01683936 -0.01428225 -0.01246879  0.01984325 -0.01571218
  0.1643929  -0.02203091  0.06600589  0.00990127]


3
and
[ 0.00615885 -0.03926127 -0.06046499  0.02653678 -0.06385841 -0.02658834
  0.07089426  0.00149316 -0.03113477 -0.07234669  0.04804381 -0.01374529
  0.07966054 -0.02819758  0.07393353 -0.01866182]


4
a
[-0.04968067 -0.07900311 -0.03253103 -0.02865176 -0.04382592 -0.02021485
  0.06473014 -0.02922015 -0.01932734  0.01414915  0.02320653 -0.0177891
  0.10318078  0.00070054  0.06550463 -0.05303453]


5
of
[-0.02960653 -0.02126354  0.01477587  0.02516394 -0.00815222  0.01286971
  0.13881849 -0.05217173 -0.02588199 -0.01729873 -0.03260932 -0.02948588
  0.06457498 -0.00467016  0.0272647

In [35]:
try:
    from google.colab import files
except ImportError:
    pass
else:
    files.download('vecs.tsv')
    files.download('meta.tsv')

In [36]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1430], [968], [4], [1537], [1537], [4739], [], [790], [2015], [11], [2922], [2191], [], [790], [2015], [11], [579], [], [11], [579], [], [4], [1783], [4], [4508], [11], [2922], [1277], [], [], [2015], [1005], [2922], [968], [579], [790], []]
